In [16]:
# Import python modules

import pandas as pd
import glob
import datetime
from datetime import timedelta, datetime, date
import requests
import os
import json
from pandas.io.json import json_normalize

from dotenv import load_dotenv
load_dotenv()

import coinbasepro as cbp
from coinbase.wallet.client import Client
import numpy as np

%matplotlib inline

In [17]:
# Read API key from .env file

cb_public_key = os.getenv("CB_PUBLIC_KEY")
cb_secret_key = os.getenv("CB_SECRET_KEY")
client = Client(cb_public_key, cb_secret_key, api_version='YYYY-MM-DD')

In [18]:
# Format date from existing ISO8601 format in CoinbasePro API using datetime function strptime formatter and conduct timeframe calculation using timedelta 

startDate = "2020-01-05T12:00:00"
start = datetime.strptime(startDate, '%Y-%m-%dT%H:%M:%S')

finalDate1 = start + timedelta(300)
finalDate2 = start - timedelta(65)
finalDate1 = finalDate1.isoformat()

finalDate1
print('-----------')
finalDate2

-----------


datetime.datetime(2019, 11, 1, 12, 0)

In [19]:
# Create a function to pull one year data from CoinBase PRO API


def datapull(startDay, endDay, myCrypto):
    
    params = {"start" : startDay, "end" : endDay, "granularity" : "86400"}
    
    url = (f" https://api.pro.coinbase.com/products/{myCrypto}-USD/candles")

    requests.get(url)
    response_data = requests.get(url, params=params)
    response_content = response_data
    print(response_content)
    data = response_data.json()
        

    data_df = pd.DataFrame(data)
    data_df.columns = ["Date", "Low", "High", "Open", "Close", "Volume"]
    data_df["Date"]= pd.to_datetime(data_df['Date'],unit='s')
    
    return data_df

# Select crypto ticker to pull data for (this only needs to be changed to get final data output. If need to pull data for different timeframe change dates below). Because CoinbasePro allows only 300 datapoints in one pull, multiple pulls with data adjustment may be needed.

crypto ="BTC"

# Adjust dates for the timeframe you need data for.

batch1 = datapull(startDate,finalDate1,crypto)
batch2 = datapull(finalDate2,startDate,crypto)

batch_combined_df = pd.concat([batch1,batch2], axis="rows", join="inner")

# Check for duplicate rows

batch_combined_df.drop_duplicates()

# Select one year data

batch_combined_df.insert(0, "Crypto", f"{crypto}")

batch_combined_df = batch_combined_df.iloc[0:365]

# Print output to csv file (enable code if need to print to .csv)

#batch_combined_df.to_csv(f"output_batch_combined_{crypto}.csv")

# Print crypto data in pandas dataframe format

batch_combined_df.tail()


<Response [200]>
<Response [200]>


,Crypto,Date,Low,High,Open,Close,Volume
60,BTC,2019-11-06,9253.66,9450.00,9322.09,9341.27,6829.686118
61,BTC,2019-11-05,9169.84,9474.89,9412.63,9322.09,7581.216673
62,BTC,2019-11-04,9127.00,9600.00,9206.52,9412.53,8361.958777
63,BTC,2019-11-03,9068.30,9380.00,9308.52,9206.52,4294.500414
64,BTC,2019-11-02,9210.00,9393.00,9253.09,9308.52,4559.265388


In [20]:
# Read multiple crypto data .csv files and concat into a dataframe

crypto_data = "../data/crypto_data"
csvfiles = glob.glob(os.path.join(crypto_data, "*csv"))

# Create a list to hold individual crypto dfs

crypto_df =[]

# Loop through .csv files and append to a crypto_df list

for csvfile in csvfiles:
    df = pd.read_csv(csvfile)
    crypto_df.append(df)
    

# Concat individual dfs into a combined df 

all_cryto_data_df = pd.concat(crypto_df, join="inner", axis="rows")

# Create pivot table to analyse Close prices for each crypto and set date as index

all_crypto_pivoted = all_cryto_data_df.pivot_table(index='Date',values='Close',columns='Crypto')

all_crypto_pivoted.head()

Crypto,BCH,BTC,DASH,EOS,ETH,LTC,REP,XLM,XRP
Date,,,,,,,,,
2019-11-01,278.98,9253.12,72.031,3.352,183.19,58.34,9.12,0.069215,0.2924
2019-11-02,290.04,9308.52,72.415,3.327,183.25,58.38,9.97,0.071528,0.2951
2019-11-03,291.64,9206.52,71.389,3.274,181.73,58.34,10.30,0.068697,0.2906
2019-11-04,291.13,9412.53,72.703,3.443,186.25,61.41,12.23,0.079496,0.2994
2019-11-05,293.30,9322.09,74.590,3.625,188.88,63.16,12.17,0.082612,0.3010
